# Random Forest - Decision Tree for neonathal deaths analysis in Colombia

## Libraries importation

In this cell are being imported all the libraries that are going to be used in the notebook. Also is imported the library warnings in order to hide the code warnings and make the notebook more readable

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import pandas as pd
import numpy as np
np.seterr(all="ignore")
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import tree
from sklearn import ensemble
from sklearn.model_selection import train_test_split

import ipywidgets as wg
from IPython.display import display, Image, HTML
import random

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='

In [3]:
def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

The following notebook is showing a preliminary analysis of [Neonatal deaths in Colombia](https://www.dane.gov.co/index.php/estadisticas-por-tema/salud/nacimientos-y-defunciones/defunciones).

The analysis is doing by using:
- Decision tree
- Random forest

In order to identify the more important socioeconomic variables that are involved in preventing neonatal deaths

## Theory

### Decision tree

A supervised learning algorithm which goal is to classify a set of items according to some independent variables values. These independent variables are called observations and are represented in the branches of the tree and the final classification or the conclusions about the item's target value are represented in the leaves of the tree.

Usually, the decision trees are used to represent decisions and decision-making in a more visually and explicitly way.

#### Example of a Decision tree

In [4]:
button1 = wg.Button(disabled=False)
button1.description = "Display image"

def action_of_the_button(b): #When one clicks the button the instance of the Button is passd to the function
    display(Image("decision_tree_example.jpg",  width = 400, height = 200))
    b.description = "Image displayed"
    b.disabled = True
    
button1.on_click(action_of_the_button)
display(button1)
hide_toggle()

Button(description='Display image', style=ButtonStyle())

The principal advantages are:
   1. They are very simple to interpret.
   2. Allow us to extract information about each of the independent variables.
   3. Can be constructed using a small amount of data.

On the other hand, the principal disadvantages are:
   1. They are inaccurate when we compare them with other classification methods.
   2. They require the introduction of several external parameters so the result depends highly on the initial conditions. 
   3. Usually overfit the training set.

To avoid these problems one possible solution is to use another, more robust method, as is random forest:

### Random Forest

In order to solve the initialization problem of the Decision Trees, the random forest constructs a multitude of decision trees with different initializations in order to get to multiple, different, decisions trees, and finally, it takes the majority voting of the decision trees. In this way, it avoids overfitting and the sensitivity to the initial conditions can be avoided.

In [5]:
button2 = wg.Button(disabled=False)
button2.description = "Display image"

def action_of_the_button(b): #When one clicks the button the instance of the Button is passd to the function
    display(Image("Random_forest_diagram_complete.png",  width = 400, height = 200))
    b.description = "Image displayed"
    b.disabled = True
    
button2.on_click(action_of_the_button)
display(button2)
hide_toggle()

Button(description='Display image', style=ButtonStyle())

## Data preparation

The dataset used already has all categorical variables as dummies in order to be used for classification. 

In [6]:
data_dummies = pd.read_csv("DataRegresionDummies.csv")

Let's see how is the dataset by looking at some example rows.

In [7]:
data_dummies.head(10)

Unnamed: 0  NACIDO_VIVO_INV  NUMERO_EMBARAZOS  SEXO_CAT  \
0     1517086                0               1.0         1   
1      631514                0               1.0         2   
2      230465                0               1.0         1   
3     1126177                0               4.0         1   
4     1544506                0               1.0         2   
5      998446                0               2.0         2   
6      571006                0               1.0         1   
7     1204297                0               3.0         2   
8      729031                0               1.0         2   
9     1302846                0               2.0         1   

   TIEMPO_GESTACION_CAT  AREA_CAT  ASIS_MED_CAT  REGION_RES_CAT  Intercept  \
0                     3         1           9.0               3          1   
1                     3         1           9.0               1          1   
2                     4         1           9.0               3          1   
3                     4         1           9.0               2          1   
4                     3         1           9.0               1          1   
5                     4         1           9.0               1          1   
6                     3         1           9.0               1          1   
7                     4         1           9.0               2          1   
8                     3         1           9.0               1          1   
9                     4         1           9.0               2          1   

   SEG_SOCIAL_CAT_2  ...  DEPTO_RESIDENCIA_76  DEPTO_RESIDENCIA_81  \
0                 0  ...                    0                    0   
1                 0  ...                    0                    0   
2                 0  ...                    1                    0   
3                 1  ...                    0                    0   
4                 0  ...                    0                    0   
5                 0  ...                    0                    0   
6                 0  ...                    0                    0   
7                 0  ...                    0                    0   
8                 0  ...                    0                    0   
9                 0  ...                    0                    0   

   DEPTO_RESIDENCIA_85  DEPTO_RESIDENCIA_86  DEPTO_RESIDENCIA_88  \
0                    0                    0                    0   
1                    0                    0                    0   
2                    0                    0                    0   
3                    0                    0                    0   
4                    0                    0                    0   
5                    0                    0                    0   
6                    0                    0                    0   
7                    0                    0                    0   
8                    0                    0                    0   
9                    0                    0                    0   

   DEPTO_RESIDENCIA_91  DEPTO_RESIDENCIA_94  DEPTO_RESIDENCIA_95  \
0                    0                    0                    0   
1                    0                    0                    0   
2                    0                    0                    0   
3                    0                    0                    0   
4                    0                    0                    0   
5                    0                    0                    0   
6                    0                    0                    0   
7                    0                    0                    0   
8                    0                    0                    0   
9                    0                    0                    0   

   DEPTO_RESIDENCIA_97  DEPTO_RESIDENCIA_99  
0                    0                    0  
1                    0                    0  
2                    0                    0  
3       

Now is important to select the variables that we think could allow us to predict when a baby could born dead. 

In [8]:
#Select the features and the target value
X = data_dummies.drop(['Unnamed: 0','NACIDO_VIVO_INV','ASIS_MED_CAT','TIEMPO_GESTACION_CAT','SEXO_CAT','AREA_CAT','REGION_RES_CAT','Intercept'],axis=1)
y = data_dummies['NACIDO_VIVO_INV']

Then, in order to evaluate our model using external data we separate the X and y sets into two subsets each in order to perform the following algorithm:
   1. The X_train will be used to train the model in order to classify correctly the most amount of y_train. This will do it by using the library sklearn.  
   2. Use the model trained before to predict the values of $\hat{y}$ for the subset X_test.
   3. Compare the values predicted ($\hat{y}$) and the real values (y_test) in order to obtain the performance of the model.
   
This has to be done for both models (Random Forest and Decision Tree).

In [9]:
#Separate between train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

## Random Forest

### ROC curve

**Receiver operating characteristic curve**:

Is a measure of the True positives as a function of the False positives. The perfect model will obtain a value of $AUC = 1.0$, and a random guess will obtain $AUC = 0.5$.

First, we need to train the model and calculate the ROC curve values and the AUC metric

In [10]:
#Create the model (Random forest)
born_forest = ensemble.RandomForestClassifier(n_estimators=100, max_depth=12)
born_forest.fit(X_train, y_train)

#Plot the ROC curve and calculate the AUC
pscore_forest = born_forest.predict_proba(X_train)[:,1]

roc_p = metrics.roc_curve(y_train, pscore_forest)
auc_p = auc_p = metrics.auc(roc_p[0],roc_p[1] )

hide_toggle()

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish t

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

In [11]:
def plot_ROC_curve_random_forest():
    line_width = 2
    plt.plot(roc_p[0], roc_p[1], color='darkorange', lw=line_width,
              label=f"ROC curve for Random Forest Classifier(AUC = {auc_p:0.2f})")
    plt.plot([0, 1], [0, 1], color='navy', lw=line_width, linestyle='--', label='Random guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title("Classification of Defaulters")
    plt.legend(loc='lower right')

button3 = wg.Button(disabled=False)
button3.description = "Display image"
def display_image(b): 
    plot_ROC_curve_random_forest()
    b.description = "Image displayed"
    b.disabled = True
button3.on_click(display_image)
display(button3)

button4 = wg.Button(disabled=False)
button4.description = "Save image"
def save_image(b): 
    if button3.disabled == False:
        plot_ROC_curve_random_forest()
        button3.description = "Image displayed"
        button3.disabled = True
    plt.savefig("confusion_matrix_RandomForest.png",bbox_inches = 'tight')
    b.description = "Image saved"
    b.disabled = True
button4.on_click(save_image)
display(button4)

hide_toggle()

Button(description='Display image', style=ButtonStyle())

Button(description='Save image', style=ButtonStyle())

### Feature importances

The random forest algorithm allows us to obtain information about how important are the different features for the classification. 

In our problem, this is very important since our major goal is to identify the socio-economical variables that affect the most the probability of a baby to born death.

In [12]:
#Calculate the feature importance as the sum of the importance of every category 
#belonging to each feature
featureImportances = {'NUMERO_EMBARAZOS':0,'SEG_SOCIAL':0,'NIV_EDUCATIVO_M':0,\
                      'TIPO_EMBARAZO':0,'AREA_RESIDENCIA':0,'DEPTO_RESIDENCIA':0}
featureImportances['NUMERO_EMBARAZOS'] = born_forest.feature_importances_[0].sum()
featureImportances['SEG_SOCIAL'] = born_forest.feature_importances_[1:6].sum()
featureImportances['NIV_EDUCATIVO_M'] = born_forest.feature_importances_[6:15].sum()
featureImportances['TIPO_EMBARAZO'] = born_forest.feature_importances_[15:19].sum()
featureImportances['AREA_RESIDENCIA'] = born_forest.feature_importances_[19:23].sum()
featureImportances['DEPTO_RESIDENCIA'] = born_forest.feature_importances_[23::].sum()

hide_toggle()

Plot the feature importance as a barplot using more natural names for each feature

In [13]:
#Plot the feature importance as a barplot using more natural names for each feature
def plot_importance_features():
    features_name = ['Number of Pregnancies','Social Security',"Mother's Educational level",\
                 'Type of Pregnancy','Residence Area','Residence Department']
    y = list(featureImportances.values())
    #plot
    fig, ax = plt.subplots(figsize=(7,4)) 
    width = 0.4 # the width of the bars 
    ind = np.arange(len(y)) # the x locations for the groups
    ax.barh(ind, y, width,color=[0,0.65490196,0.57647059])
    ax.set_yticks(ind+width/10)
    ax.set_yticklabels(features_name, minor=False)
    ax.set_xlim(0,0.8)
    for index, value in enumerate(y):
        plt.text(value+0.005, index-0.075, str(round(value,2)))

    plt.title('Feature importance in RandomForest Classifier')
    plt.xlabel('Relative importance')
    plt.tight_layout()

button5 = wg.Button(disabled=False)
button5.description = "Display image"
def display_image(b): 
    plot_importance_features()
    b.description = "Image displayed"
    b.disabled = True
button5.on_click(display_image)
display(button5)

button6 = wg.Button(disabled=False)
button6.description = "Save image"
def save_image(b): 
    if button5.disabled == False:
        plot_importance_features()
        button5.description = "Image displayed"
        button5.disabled = True
    plt.savefig("feature_importance.png",bbox_inches = 'tight')
    b.description = "Image saved"
    b.disabled = True
button6.on_click(save_image)
display(button6)

hide_toggle()

Button(description='Display image', style=ButtonStyle())

Button(description='Save image', style=ButtonStyle())

In order to evaluate the performance of the model, now we plot the confusion matrix for the test set

In [14]:
def plot_confusion_matrix():
    pred = np.array(born_forest.predict(X_test) > 0.5, dtype=float)
    cm = np.histogram2d(y_test, pred, bins=2)[0]

    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True,annot_kws={"size": 20})
    # labels, title and ticks
    plt.xlabel('Predicted values',fontsize=18)
    plt.ylabel('Real values',fontsize=18)
    plt.title('Confusion matrix',fontsize=24)
    plt.tight_layout()

button7 = wg.Button(disabled=False)
button7.description = "Display image"
def display_image(b): 
    plot_confusion_matrix()
    b.description = "Image displayed"
    b.disabled = True
button7.on_click(display_image)
display(button7)

button8 = wg.Button(disabled=False)
button8.description = "Save image"
def save_image(b): 
    if button7.disabled == False:
        plot_confusion_matrix()
        button7.description = "Image displayed"
        button7.disabled = True
    plt.savefig("confusion_matrix_RandomForest.png",bbox_inches = 'tight')
    b.description = "Image saved"
    b.disabled = True
button8.on_click(save_image)
display(button8)

hide_toggle()

Button(description='Display image', style=ButtonStyle())

Button(description='Save image', style=ButtonStyle())

## Decision Tree

### ROC curve

In [15]:
#Create the model (Decision Tree)
born_tree = tree.DecisionTreeClassifier(max_depth = 3)
born_tree = born_tree.fit(X_train, y_train)

#Plot the ROC curve and calculate the AUC
pscore_tree = born_tree.predict_proba(X_train)[:,1]

roc_p = metrics.roc_curve(y_train, pscore_tree)
auc_p = auc_p = metrics.auc(roc_p[0],roc_p[1] )
hide_toggle()

/home/morales/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


In [16]:
def plot_ROC_curve_decision_tree():
    line_width = 2
    plt.plot(roc_p[0], roc_p[1], color='darkorange', lw=line_width,
              label=f"ROC curve for Random Forest Classifier(AUC = {auc_p:0.2f})")
    plt.plot([0, 1], [0, 1], color='navy', lw=line_width, linestyle='--', label='Random guess')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title("Classification of Defaulters")
    plt.legend(loc='lower right')

button9 = wg.Button(disabled=False)
button9.description = "Display image"
def display_image(b): 
    plot_ROC_curve_decision_tree()
    b.description = "Image displayed"
    b.disabled = True
button9.on_click(display_image)
display(button9)

button10 = wg.Button(disabled=False)
button10.description = "Save image"
def save_image(b): 
    if button9.disabled == False:
        plot_ROC_curve_decision_tree()
        button9.description = "Image displayed"
        button9.disabled = True
    plt.savefig("roc_curve_decision_tree.png",bbox_inches = 'tight')
    b.description = "Image saved"
    b.disabled = True
button10.on_click(save_image)
display(button10)

hide_toggle()

Button(description='Display image', style=ButtonStyle())

Button(description='Save image', style=ButtonStyle())

As it waas said before, the decision tree allow us to see the clasificator in a more natural way, its even possible to plot and analyze the tree obtained after the training.

In [17]:
#Plot the tree
def plot_tree():
    fig = plt.gcf()
    fig.set_size_inches(150, 100)
    tree.plot_tree(born_tree,filled=True,feature_names=list(X_train.columns))

button11 = wg.Button(disabled=False)
button11.description = "Display image"
def display_image(b): 
    plot_ROC_curve_decision_tree()
    b.description = "Image displayed"
    b.disabled = True
button11.on_click(display_image)
display(button11)

button12 = wg.Button(disabled=False)
button12.description = "Save image"
def save_image(b): 
    if button11.disabled == False:
        plot_ROC_curve_decision_tree()
        button11.description = "Image displayed"
        button11.disabled = True
    plt.savefig("tree_decision_tree.png",bbox_inches = 'tight')
    b.description = "Image saved"
    b.disabled = True
button12.on_click(save_image)
display(button12)

hide_toggle()

Button(description='Display image', style=ButtonStyle())

Button(description='Save image', style=ButtonStyle())

In [18]:
pred = np.array(born_tree.predict(X_test) > 0.5, dtype=float)
cm = np.histogram2d(y_test, pred, bins=2)[0]
hide_toggle()

In [19]:
#Plot the confusion matrix for the test set
def plot_confusion_matrix_decision_tree():
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True,annot_kws={"size": 20})
    # labels, title and ticks
    plt.xlabel('Predicted values',fontsize=18)
    plt.ylabel('Real values',fontsize=18)
    plt.title('Confusion matrix',fontsize=24)
    plt.tight_layout()

button13 = wg.Button(disabled=False)
button13.description = "Display image"
def display_image(b): 
    plot_confusion_matrix_decision_tree()
    b.description = "Image displayed"
    b.disabled = True
button13.on_click(display_image)
display(button13)

button14 = wg.Button(disabled=False)
button14.description = "Save image"
def save_image(b): 
    if button13.disabled == False:
        plot_confusion_matrix_decision_tree()
        button13.description = "Image displayed"
        button13.disabled = True
    plt.savefig("tree_decision_tree.png",bbox_inches = 'tight')
    b.description = "Image saved"
    b.disabled = True
button14.on_click(save_image)
display(button14)

hide_toggle()

Button(description='Display image', style=ButtonStyle())

Button(description='Save image', style=ButtonStyle())